### Import file and lib

In [205]:
import pandas as pd

df = pd.read_json("fallacies.json", lines=True)
df

,name,description,logical_form,explanation_with_examples,article
0,Accent Fallacy,"When the meaning of a word, sentence, or entir...",Claim is made with accent on word X giving cla...,"Example #1: In the movie, My Cousin Vinny, Ral...","The description of the fallacy of type ""Accent..."
1,Accident Fallacy,When an attempt is made to apply a general rul...,"X is a common and accepted rule. Therefore, th...",Example #1: I believe one should never deliber...,"The description of the fallacy of type ""Accide..."
2,Ad Fidentia,Attacking the person’s self-confidence in plac...,"Person 1 claims that Y is true, but is person ...",Example #1: Rick: I had a dream last night tha...,"The description of the fallacy of type ""Ad Fid..."
3,Ad Hoc Rescue,Very often we desperately want to be right and...,Claim X is true because of evidence Y. Evidenc...,Example #1: Frieda: I just know that Raymond i...,"The description of the fallacy of type ""Ad Hoc..."
4,Ad Hominem Abusive,"Attacking the person making the argument, rath...",Person 1 is claiming Y. Person 1 is a moron. T...,Example #1: My opponent suggests that lowering...,"The description of the fallacy of type ""Ad Hom..."
...,...,...,...,...,...
229,Unwarranted Contrast,"Assuming that implicature means implication, w...","Some S are P. Therefore, some S are not P. ...",Example #1: Some atheists are human. Therefore...,"The description of the fallacy of type ""Unwarr..."
230,Use Mention Error,"Confusing the word used to describe a thing, w...",“X” is the same as X.,Example #1: My son is made up of five letters....,"The description of the fallacy of type ""Use Me..."
231,Weak Analogy,When an analogy is used to prove or disprove a...,"X is like Y. Y has property P. Therefore, X ha...",Example #1: Not believing in the literal resur...,"The description of the fallacy of type ""Weak A..."
232,Willed Ignorance,Refusing to change one’s mind or consider conf...,I believe X. You have evidence for Y. I don’t ...,Example #1: I don’t want anything coming in th...,"The description of the fallacy of type ""Willed..."


### Extract labels

In [207]:
true_labels = df.name.tolist()
print(true_labels)

['Accent Fallacy', 'Accident Fallacy', 'Ad Fidentia', 'Ad Hoc Rescue', 'Ad Hominem Abusive', 'Ad Hominem Circumstantial', 'Ad Hominem Guilt by Association', 'Ad Hominem Tu quoque', 'Affirmative Conclusion from a Negative Premise', 'Affirming a Disjunct', 'Affirming the Consequent', 'Alleged Certainty', 'Alphabet Soup', 'Alternative Advance', 'Amazing Familiarity', 'Ambiguity Fallacy', 'Anonymous Authority', 'Anthropomorphism', 'Appeal to Accomplishment', 'Appeal to Anger', 'Appeal to Authority', 'Appeal to Celebrity', 'Appeal to Closure', 'Appeal to Coincidence', 'Appeal to Common Belief', 'Appeal to Common Folk', 'Appeal to Common Sense', 'Appeal to Complexity', 'Appeal to Consequences', 'Appeal to Definition', 'Appeal to Desperation', 'Appeal to Emotion', 'Appeal to Equality', 'Appeal to Extremes', 'Appeal to Faith', 'Appeal to False Authority', 'Appeal to Fear', 'Appeal to Flattery', 'Appeal to Force', 'Appeal to Heaven', 'Appeal to Intuition', 'appeal to loyalty', 'Appeal to Nature

### Extract Examples

In [209]:
#Extract the part before the first "Explanation: " as the example, so basically to extract example 1. But due to format errors in the dataset it doesn't always work out
separator = 'Explanation: '
examples = df.explanation_with_examples.astype(str).str.split(separator, n = 1).str[0].tolist()


### Face Validity Check by checking the length of data

In [211]:
print(len(true_labels))
print(len(examples))

234
234


### Prompt Template

In [213]:
first_level_prompt_template = """
I will ask you to classify the specific type of logical fallacies in the example, you will get different reward for getting the correct answer on different levels, for example, if you are able to get the correct classification on level 1, you get reward 1, otherwise you get 0. If you are able to get the correct classification on level 1, you move on to the next level, and if you are able to get the correct classification on level 2, you get reward 2, if you are able to get the correct classification on level 3, you get reward 3, and so on. The first level of classification challenge is to classify if the logical fallacy in the statement is formal or informal. The definition of formal fallacy means that there is an error in the argument's form. In contrast, informal fallacy means the arguments are logically unsound for lack of well-grounded premises. Now classify the below statement and output the results in this format: "level_1_results:#{}#" in all lower cases with no "fallacy". Only output the level 1 classification, no explanations.
"""
second_level_prompt_template = """
Now you are moved to level 2 of classification, where you need to further classify the fallacy. Under formal fallacy, there are proposition(Errors in dealing with the logical relations holding between propositions.), Quantification Fallacy (Errors in dealing with the quantifiers), Syllogism Fallacy (Errors in the syllogisms of deductive reasoning.), Probability Fallacy (Errors in dealing with probability.). Under informal fallacy, there are Ambiguity Fallacy (Errors due to linguistic ambiguity or vagueness of terms.), Inconsistency Fallacy (Self-contradiction and inconsistency occurs.), Irrelevance Fallacy (The premises are irrelevant to the conclusion.), Insufficiency Fallacy (The premises are insufficient or weak to support the conclusion.) and Inappropriate Presumption Fallacy (An inappropriate presumption is explicitly or implicitly introduced.), which one do you think it would fall under? You can also go back and change your previous classification. Output the results in this format: "level_2_results:#{}#" in all lower cases with no "fallacy". Only output the level 2 classification, no explanations. Only output a classification from the level 2 fallacy list provided in the format requested.
"""
third_level_prompt_template_syllogism = """
Moving on to level three, under Syllogism, there are Fallacy of the Undistributed Middle (A formal fallacy in a categorical syllogism where the middle term, or the term that does not appear in the conclusion, is not distributed to the other two terms.), Exclusive Premises (A standard form categorical syllogism that has two negative premises either in the form of “no X are Y” or “some X are not Y”.), Fallacy of Four Terms (This fallacy occurs in a categorical syllogism when the syllogism has four terms rather than the requisite three.), Illicit Substitution of Identicals (A fallacy due to confusing the knowing of a thing (extension) with the knowing of it under all its various names or descriptions (intension).), Illicit Minor (Any form of a categorical syllogism in which the minor term is distributed in the conclusion, but not in the minor premise.), Illicit Major (Any form of a categorical syllogism in which the major term is distributed in the conclusion, but not in the major premise.), Negative Conclusion from Affirmative Premises (The conclusion of a standard form categorical syllogism is negative, but both of the premises are positive.), Affirmative Conclusion from a Negative Premise (The conclusion of a standard form categorical syllogism is affirmative, but at least one of the premises is negative). Which one do you think it should fall under? You can also go back and change your previous classification. Output the results in this format: "level_3_results:#{}#" in all lower cases with no "fallacy". Only output the level 3 classification, no explanations. Only output a classification from the level 3 fallacy list provided in the format requested.
"""
third_level_prompt_template_proposition = """
Moving on to level three, under Proposition, there are Affirming the Consequent (An error in formal logic where if the consequent is said to be true, the antecedent is said to be true, as a result.), Denying the Antecedent (It is a fallacy where in a standard if/then premise, the antecedent (what comes after the “if”) is made not true, then it is concluded that the consequent (what comes after the “then”) is not true.), Negating Antecedent and Consequent (The valid form of this argument is as follows: If P then Q. Therefore, if not-Q then not-P.), Commutation of Conditionals (Switching the antecedent and the consequent in a logical argument.), Affirming a Disjunct (Making the false assumption that when presented with an either/or possibility, that if one of the options is true that the other one must be false.), Denying a Conjunct (A formal fallacy in which the first premise states that at least one of the two conjuncts (antecedent and consequent) is false and concludes that the other conjunct must be true.)  Which one do you think it should fall under? You can also go back and change your previous classification. Output the results in this format: "level_3_results:#{}#" in all lower cases with no "fallacy". Only output the level 3 classification, no explanations. Only output a classification from the level 3 fallacy list provided in the format requested.
"""
third_level_prompt_template_quantification = """
Moving on to level three, under Quantification, there are False Conversion (The formal fallacy where the subject and the predicate terms of the proposition are switched (conversion) in the conclusion, in a proposition that uses “all” in its premise (type “A” forms), or “some/not” (type “O” forms).), Unwarranted Contrast (Assuming that implicature means implication, when it logically does not.), Quantifier Shift Fallacy (A fallacy of reversing the order of two quantifiers.), Existential Fallacy (A formal logical fallacy, which is committed when a categorical syllogism employs two universal premises (“all”) to arrive at a particular (“some”) conclusion.), Fallacy of Every and All (When an argument contains both universal quantifiers and existential quantifiers (all, some, none, every) with different meanings, and the order of the quantifiers is reversed.), Illicit Contraposition (A formal fallacy where switching the subject and predicate terms of a categorical proposition, then negating each, results in an invalid argument form.) Which one do you think it should fall under? You can also go back and change your previous classification. Output the results in this format: "level_3_results:#{}#" in all lower cases with no "fallacy". Only output the level 3 classification, no explanations. Only output a classification from the level 3 fallacy list provided in the format requested.
"""
third_level_prompt_template_probability = """
Moving on to level three, under probability, there are Gamblers Fallacy (Reasoning that, in a situation that is pure random chance, the outcome can be affected by previous outcomes), Hot Hand Fallacy (The irrational belief that if you win or lose several chance games in a row, you are either “hot” or “cold,” respectively, meaning that the streak is likely to continue and has to do with something other than pure probability), Conjunction Fallacy (This occurs when one estimates a conjunctive statement (this and that) to be more probable than at least one of its component statements.), Disjunction Fallacy (It occurs when one estimates a disjunctive statement (this or that) to be less probable than at least one of its component statements.) Which one do you think it should fall under? You can also go back and change your previous classification. Output the results in this format: "level_3_results:#{}#" in all lower cases with no "fallacy". Only output the level 3 classification, no explanations. Only output a classification from the level 3 fallacy list provided in the format requested.
"""
third_level_prompt_template_ambiguity = """
Moving on to level three, under Ambiguity Fallacy, there are Argument of the Beard Fallacy (When one argues that no useful distinction can be made between two extremes, just because there is no definable moment or point on the spectrum where the two extremes meet.), Appeal to Extremes Fallacy (Erroneously attempting to make a reasonable argument into an absurd one, by taking the argument to the extremes.), Type Token Fallacy (The type-token fallacy is committed when a word can refer to either a type (representing an abstract descriptive concept) or a token (representing an object that instantiates a concept) and is used in a way that makes it unclear which it refers to.), Use Mention Error Fallacy (Confusing the word used to describe a thing, with the thing itself.), Reification Fallacy (When an abstraction is treated as if it were a concrete, real event or physical entity – when an idea is treated as if had a real existence.), Fake Precision Fallacy (Using implausibly precise statistics to give the appearance of truth and certainty, or using a negligible difference in data to draw incorrect inferences.), No True Scotsman Fallacy (When a universal (“all”, “every”, etc.) claim is refuted, rather than conceding the point or meaningfully revising the claim, the claim is altered by going from universal to specific, and failing to give any objective criteria for the specificity.), Contextomy Fallacy (Removing a passage from its surrounding matter in such a way as to distort its intended meaning.), Stolen Concept Fallacy (Requiring the truth of the something that you are simultaneously trying to disprove), Anthropomorphism Fallacy (The attributing of human characteristics and purposes to inanimate objects, animals, plants, or other natural phenomena, or to gods.), Accent Fallacy (When the meaning of a word, sentence, or entire idea is interpreted differently by changing where the accent falls.), Ambiguity Fallacy (When an unclear phrase with multiple definitions is used within the argument; therefore, does not support the conclusion.), Alphabet Soup Fallacy (The deliberate and excessive use of acronyms and abbreviations to appear more knowledgeable in the subject or confuse others.), Equivocation Fallacy (Using an ambiguous term in more than one sense, thus making an argument misleading.) and Modal Scope Fallacy (Making a formal argument invalid by confusing the scope of what is actually necessary or possible.), which one do you think it would fall under? You can also go back and change your previous classification. Output the results in this format: "level_3_results:#{}#" in all lower cases with no "fallacy". Only output the level 3 classification, no explanations. Only output a classification from the level 3 fallacy list provided in the format requested.
"""
third_level_prompt_template_inconsistency = """
Moving on to level three, under Inconsistency, there are Inconsistency (In terms of a fallacious argument, two or more propositions are
asserted that cannot both possibly be true.), Conflicting Conditions (When the argument is self-contradictory and cannot possibly be true.), and Kettle Logic (Making (usually) multiple, contradicting arguments, in an attempt to support a single point or idea.). Which one do you think it should fall under? You can also go back and change your previous classification. Output the results in this format: "level_3_results:#{}#" in all lower cases with no "fallacy". Only output the level 3 classification, no explanations. Only output a classification from the level 3 fallacy list provided in the format requested.
"""
third_level_prompt_template_irrelevance = """
Moving on to level three, under Irrelevance, there are Political Correctness Fallacy (It is the assumption or admission that two or more groups, individuals, or ideas of groups or individuals, are equal, of equal value, or both true, based on the recent phenomenon of political correctness), Appeal to Complexity Fallacy (Concluding that because you don’t understand something, it must not be true, it’s improbable, or the argument must be flawed.), Statement of Conversion Fallacy (Accepting the truth of a claim based on a conversion story without considering any evidence for the truth of the claim.), Appeal to the Moon Fallacy (Using the argument, “If we can put a man on the moon, we could...” as evidence for the argument.), Quantum Physics Fallacy (Using quantum physics in an attempt to support your claim, when in no way is your claim related to quantum physics.), fact to fiction fallacy (Attempting to support a narrative or argument with facts that don’t support the narrative or argument.), Non Sequitur Fallacy (Evidence or reason is irrelevant or adds very little support to the conclusion.), Inflation of Conflict (Reasoning that because authorities cannot agree precisely on an issue, no conclusions can be reached at all, and minimizing the credibility of the authorities, as a result.), Argument by Fast Talking (When fast talking is seen as intelligence and/or confidence in the truth of one’s argument; therefore, seen as evidence of the truth of the argument itself.), Appeal to Intuition (Evaluating an argument based on "intuition" or "gut feeling" that is unable to be articulated, rather than evaluating the argument using reason.), Appeal to Closure (Accepting evidence on the basis of wanting closure, or to be done with the issue.), Appeal to Definition (Using a dictionary’s limited definition of a term as evidence that term cannot have another meaning, expanded meaning, or even conflicting meaning.), Spiritual Fallacy (Insisting that something meant to be literal is actually “spiritual” as an explanation or justification for something that otherwise would not fit in an explanation.), gish gallop (Overwhelming an interlocutor with as many arguments as possible, without regard for accuracy or strength of the arguments.), Denying the Correlative (Introducing alternatives when, in fact, there are none.), Red Herring (Attempting to redirect the argument to another issue to which the person doing the redirecting can better respond.), Strawman Fallacy (Substituting a person’s actual position or argument with a distorted, exaggerated, or misrepresented version of the position of the argument.), Avoiding the Issue (When an arguer responds to an argument by not addressing the points of the argument), Logic Chopping (Using the technical tools of logic in an unhelpful and pedantic manner by focusing on trivial details instead of directly addressing the main issue in dispute.), Meaningless Question (Asking a question that cannot be answered with any sort of rational meaning.), Failure to Elucidate (When the definition is made more difficult to understand than the word or concept being defined.), Argument by Gibberish (When incomprehensible jargon or plain incoherent gibberish is used to give the appearance of a strong argument, in place of evidence or valid reasons to accept the argument.), Hypnotic Bait and Switch (Stating several uncontroversially true statements in succession, followed by a claim that the arguer wants the audience to accept as true.), Traitorous Critic Fallacy (Responding to criticism by attacking a person’s perceived favorability to an out-group or dislike to the in-group as the underlying reason for the criticism rather than addressing the criticism itself, and suggesting that they stay away from the issue and/or leave the in-group.), Having Your Cake (Making an argument, or responding to one, in such a way that it does not make it at all clear what your position is.), Appeal to Common Belief (When the claim that most or many people in general or of a particular group accept a belief as true is presented as evidence for the claim.), Appeal to Popularity (Using the popularity of a premise or proposition as evidence for its truthfulness.), Appeal to Common Sense (Asserting that your conclusion or facts are just “common sense” when, in fact, they are not.), Appeal to Common Folk (In place of evidence, attempting to establish a connection to the audience based on being a “regular person” just like each of them.), Appeal to Trust (The belief that if a source is considered trustworthy or untrustworthy, then any information from that source must be true or false, respectively.), Argument from Age (The misconception that previous generations had superior wisdom to modern man, thus conclusions that rely on this wisdom are seen accepted as true or more true than they actually are.), Appeal to Heaven (Asserting the conclusion must be accepted because it is the “will of God” or “the will of the gods”.), Appeal to Tradition (Using historical preferences of the people (tradition), either in general or as specific as the historical preferences of a single individual, as evidence that the historical preference is correct.), Etymological Fallacy (The assumption that the present-day meaning of a word should be/is similar to the historical meaning.), Genetic Fallacy (Basing the truth claim of an argument on the origin of its claims or premises.), Appeal to Celebrity (Accepting a claim of a celebrity based on his or her celebrity status, not on the strength of the argument.), Appeal to Authority (Insisting that a claim is true simply because a valid authority or expert on the issue said it was true, without any other supporting evidence offered.), Appeal to False Authority (Using an alleged authority as evidence in your argument when the authority is not really an authority on the facts relevant to the argument.), Argument from False Authority (When a person making a claim is presented as an expert who should be trusted when his or her expertise is not in the area being discussed.), Blind Authority Fallacy (Asserting that a proposition is true solely on the authority making the claim), Argument by Personal Charm (When an argument is made stronger by the personal characteristics of the person making the argument, often referred to as “charm”.), Argument to the Purse (Concluding that the truth value of the argument is true or false based on the financial status of the author of the argument or the money value associated with the truth.), Ad Hominem Circumstantial (Suggesting that the person who is making the argument is biased or predisposed to take a particular stance, and therefore, the argument is necessarily invalid.), Gadarene Swine Fallacy (The assumption that because an individual is not in formation with the group, that the individual must be the one off course.), Ad Hominem Tu quoque (Claiming the argument is flawed by pointing out that the one making the argument is not acting consistently with the claims of the argument.), Bulverism (It is the assumption and assertion that an argument is flawed or false because of the arguer’s characteristic associated with the arguer’s identity.), Righteousness Fallacy (Assuming that just because a person’s intentions are good, they have the truth or facts on their side.), Self Righteousness Fallacy (Assuming that just because your intentions are good, you have the truth or facts on your side.), Reductio ad Hitlerum (The attempt to make an argument analogous with Hitler or the Nazi party.), Ad Hominem Guilt by Association (When the source is viewed negatively because of its association with another person or group who is already viewed negatively.), Identity Fallacy (When one’s argument is evaluated based on their physical or social identity when the strength of the argument is independent of identity.), Appeal to Stupidity (Attempting to get the audience to devalue reason and intellectual discourse, or devaluing reason and intellectual discourse based on the rhetoric of an arguer.), Ad Hominem Abusive (Attacking the person making the argument, rather than the argument itself, when the attack on the person is completely irrelevant to the argument the person is making.), Ad Fidentia (Attacking the person’s self-confidence in place of the argument or the evidence.), appeal to loyalty (When one is either implicitly or explicitly encouraged to consider loyalty when evaluating the argument when the truth of the argument is independent of loyalty.), Appeal to Accomplishment (When the argument being made is sheltered from criticism based on the level of accomplishment of the one making the argument.), Scapegoating (Unfairly blaming an unpopular person or group of people for a problem or a person or group that is an easy target for such blame.), Fallacy of Opposition (Asserting that those who disagree with you must be wrong and not thinking straight, primarily based on the fact that they are the opposition.), Proof by Intimidation (Making an argument purposely difficult to understand in an attempt to intimidate your audience into accepting it, or accepting an argument without evidence or being intimidated to question the authority or a priori assumptions of the one making the argument.), Poisoning the Well (To prime the audience with adverse information about the opponent from the start, in an attempt to make your claim more acceptable or discount the credibility of your opponent’s claim.), Wishful Thinking (When the desire for something to be true is used in place of/or as evidence for the truthfulness of the claim.), Appeal to Faith (It is the assertion that one must have (the right kind of) faith in order to understand the argument.), Notable Effort (Accepting good effort as a valid reason to accept the truth of the conclusion, even though the effort is unrelated to the truth.), Prejudicial Language (Loaded or emotive terms used to attach value or moral goodness to believing the proposition.), Special Pleading (Applying standards, principles, and/or rules to other people or circumstances, while making oneself or certain circumstances exempt from the same critical criteria, without providing adequate justification.), If By Whiskey (A response to a question that is contingent on the questioner’s opinions and makes use of words with strong connotations.), Overextended Outrage (One or more statistically rare cases are implied to be the norm or the trend (without evidence) for the purpose of expressing or inciting outrage toward an entire group.), Appeal to Ridicule (Presenting the argument in such a way that makes the argument look ridiculous, usually by misrepresenting the argument or the use of exaggeration.), Argument by Emotive Language (Substituting facts and evidence with words that stir up emotion, with the attempt to manipulate others into accepting the truth of the argument.), Style Over Substance (When the arguer embellishes the argument with compelling language or rhetoric, and/or visual aesthetics), Appeal to Anger (When the emotions of anger, hatred, or rage are substituted for evidence in an argument.), Appeal to Pity (The attempt to distract from the truth of the conclusion by the use of pity.), Appeal to Emotion (This is the general category of many fallacies that use emotion in place of reason in order to attempt to win the argument.), Appeal to Flattery (When an attempt is made to win support for an argument, not by the strength of the argument, but by using flattery on those whom you want to accept your argument.), Appeal to Spite (Substituting spite (petty ill will or hatred with the disposition to irritate, annoy, or thwart) for evidence in an argument, or as a reason to support or reject a claim.), pragmatic fallacy (Claiming that something is true because the person making the claim has experienced, or is referring to someone who has experienced, some practical benefit from believing the thing to be true.), Appeal to Force (When force, coercion, or even a threat of force is used in place of a reason in an attempt to justify a conclusion.), Appeal to Fear (When fear, not based on evidence or reason, is being used as the primary motivator to get others to accept an idea, proposition, or conclusion.). which one do you think it would fall under? You can also go back and change your previous classification. Output the results in this format: "level_3_results:#{}#" in all lower cases with no "fallacy". Only output the level 3 classification, no explanations. Only output a classification from the level 3 fallacy list provided in the format requested.
"""
third_level_prompt_template_insufficiency = """
Moving on to level three, under Insufficiency, there are Fallacy of Composition (Inferring that something is true of the whole from the fact that it is true of some part of the whole.), Fallacy of Division (Inferring that something is true of one or more of the parts from the fact that it is true of the whole.), Stereotyping the fallacy (The general beliefs that we use to categorize people, objects, and events while assuming those beliefs are accurate generalizations of the whole group.), Ecological Fallacy (The interpretation of statistical data where inferences about the nature of individuals are deduced from inference for the group to which those individuals belong.), Oversimplified Cause Fallacy (When a contributing factor is assumed to be the cause, or when a complex array of causal factors is reduced to a single cause.), Accident Fallacy (When an attempt is made to apply a general rule to all situations when clearly there are exceptions to the rule.), mcnamara fallacy (When a decision is based solely on quantitative observations (i.e.,metrics, hard data, statistics) and all qualitative factors are ignored.), Overwhelming Exception (A generalization that is technically accurate, but has one or more qualifications which eliminate so many cases that the resulting argument is significantly weaker than the arguer implies.), Reductio ad Absurdum (A mode of argumentation or a form of argument in which a proposition is disproven by following its implications logically to an absurd conclusion), Nirvana Fallacy (Comparing a realistic solution with an idealized one, and discounting or even dismissing the realistic solution.), Relative Privation (Trying to make a scenario appear better or worse by comparing it to
the best or worst case scenario.), imposter fallacy (When one suggests or claims, with insufficient evidence, that the group outliers who are viewed as damaging to the group are primarily made up of infiltrators of another group with the purpose of making the infiltrated group look bad.), Misleading Vividness (A small number of dramatic and vivid events are taken to outweigh a significant amount of statistical evidence), Appeal to Possibility (When a conclusion is assumed not because it is probably true, but because it is possible that it is true, no matter how improbable.), Rights To Ought Fallacy (When one conflates a reason for one’s rights (constitutional or other) with what one should do.), Psychogenetic Fallacy (Inferring some psychological reason why an argument is made then assuming it is invalid, as a result.), Weak Analogy (When an analogy is used to prove or disprove an argument, but the analogy is too dissimilar to be effective, that is, it is unlike the argument more than it is like the argument.), Extended Analogy (Suggesting that because two things are alike in some way and one of those things is like something else, then both things must be like that "something else".), Appeal to Equality (An assertion is deemed true or false based on an assumed pretense of equality, where what exactly is "equal" is not made clear, and not supported by the argument.), False Equivalence (An argument or claim in which two completely opposing arguments appear to be logically equivalent when in fact they are not.), Galileo Fallacy (The claim that because an idea is forbidden, prosecuted, detested, or otherwise mocked, it must be true, or should be given more credibility.), Post Designation (Drawing a conclusion from correlations observed in a given sample, but only after the sample has already been drawn, and without declaring in advance what correlations the experimenter was expecting to find.), Just In Case Fallacy (Making an argument based on the worst-case scenario rather than the most probable scenario, allowing fear to prevail over reason.), Selective Attention (Focusing your attention on certain aspects of the argument while completely ignoring or missing other parts.), nutpicking fallacy (When someone presents an atypical or weak member of a group as if they are a typical or strong representative.), Biased Sample Fallacy (Drawing a conclusion about a population based on a sample that is biased, or chosen in order to make it appear the population on average is different than it actually is.), Survivorship Fallacy (In its general form, the survivorship fallacy is basing a conclusion on a limited number of "winner" testimonies due to the fact we cannot or do not hear the testimonies of the losers.), Spotlight Fallacy (Assuming that the media’s coverage of a certain class or category is representative of the class or category in whole.), Hasty Generalization (Drawing a conclusion based on a small sample size, rather than looking at statistics that are much more in line with the typical or average situation.), Incomplete Comparison (An incomplete assertion that cannot possibly be refuted. This is popular in advertising.), Texas Sharpshooter Fallacy (Ignoring the difference while focusing on the similarities, thus coming to an inaccurate conclusion.), Faulty Comparison (Comparing one thing to another that is really not related, in order to make one thing look more or less desirable than it really is.), Base Rate Fallacy (Ignoring statistical information in favor of using irrelevant information, that one incorrectly believes to be relevant, to make a judgment.), Least Plausible Hypothesis (Choosing more unreasonable explanations for phenomena over more defensible ones.), Far Fetched Hypothesis (Offering a bizarre (far-fetched) hypothesis as the correct explanation without first ruling out more mundane explanations.), Cherry Picking (When only select evidence is presented in order to persuade the audience to accept a position, and evidence that would go against the position is withheld.), Argument by Selective Reading (When a series of arguments or claims is made and the opponent acts as if the weakest argument was the best one made.), deceptive sharing (Sharing an article, post, or meme on social media with the intent to influence public perception to perceive a statistically rare event as a common event.), Multiple Comparisons Fallacy (Claiming that unexpected trends that occur through random chance alone in a data set with a large number of variables are meaningful.), Magical Thinking (Making causal connections or correlations between two events not based on logic or evidence, but primarily based on superstition.), Slippery Slope (When a relatively insignificant first event is suggested to lead to a more significant event, which in turn leads to a more significant event, and so on, until some ultimate, significant event is reached, where the connection of each event is not only unwarranted but with each step it becomes more and more improbable.), Sunk Cost Fallacy (Reasoning that further investment is warranted on the fact that the resources already invested will be lost otherwise, not taking into consideration the overall losses involved in the further investment.), Jumping to Conclusions (Drawing a conclusion without taking the needed time to evaluate the evidence or reason through the argument.), Argument from Silence (Drawing a conclusion based on the silence of the opponent, when the opponent is refusing to give evidence for any reason.), Argument from Hearsay (Presenting the testimony of a source that is not an eyewitness to the event in question.), Anonymous Authority (When an unspecified source is used as evidence for the claim.), Insignificant Cause (An explanation that posits one minor factor, out of several that contributed, as its sole cause.), Just Because Fallacy (Refusing to respond to give reasons or evidence for a claim by stating yourself as the ultimate authority on the matter.), Appeal to the Law (When following the law is assumed to be the morally correct thing to do, without justification, or when breaking the law is assumed to be the morally wrong thing to do, without justification.), Appeal to Normality (Using social norms to determine what is good or bad.), False Effect (Claiming that the cause is true or false based on what we know about the effect in a claim of causality that has not been properly established.), Appeal to Consequences (Concluding that an idea or proposition is true or false because the consequences of it being true or false are desirable or undesirable.), Retrogressive Causation (Invoking the cause to eliminate the effect, or calling on the source to relieve the effect of the source.), Confusing Currently Unexplained with Unexplainable (Making the assumption that what cannot currently be explained is, therefore, unexplainable (impossible to explain).), Appeal to Desperation (Arguing that your conclusion, solution, or proposition is right based on the fact that something must be done, and your solution is "some-
thing."), Regression Fallacy (Ascribing a cause where none exists in situations where natural fluctuations exist while failing to account for these natural fluctuations.), Causal Reductionism (Assuming a single cause or reason when there were actually multiple causes or reasons.), Questionable Cause (Concluding that one thing caused another, simply because they are regularly associated.). Which one do you think it should fall under? You can also go back and change your previous classification. Output the results in this format: "level_3_results:#{}#" in all lower cases with no "fallacy". Only output the level 3 classification, no explanations. Only output a classification from the level 3 fallacy list provided in the format requested.
"""
third_level_prompt_template_inappropriate_presumption = """
Moving on to level three, under Inappropriate Presumption, there are Hedging (Refining your claim simply to avoid counter evidence and then acting as if your revised claim is the same as the original.), Circular Definition (A circular definition is defining a term by using the term in the definition), Homunculus Fallacy (An argument that accounts for a phenomenon in terms of the very phenomenon that it is supposed to explain, which results in an infinite regress.), Circular Reasoning (A type of reasoning in which the proposition is supported by the premises, which is supported by the proposition.), Tokenism (Interpreting a token gesture as an adequate substitute for the real thing.), Appeal to Novelty (Claiming that something that is new or modern is superior to the status quo, based exclusively on its newness.), Two Wrongs Make a Right (When a person attempts to justify an action against another person because the other person did take or would take the same action against him or her.), Appeal to Nature (When used as a fallacy, the belief or suggestion that “natural” is better than “unnatural” based on its naturalness.), Naturalistic Fallacy (When the conclusion expresses what ought to be, based only on what is, or what ought not to be, based on what is not.), Moralistic Fallacy (When the conclusion expresses what is, based only on what one believes ought to be, or what isn’t is based on what one believes ought not to be.), Suppressed Correlative (The attempt to redefine a correlative (one of two mutually exclusive options) so that one alternative encompasses the other, i.e. making one alternative impossible.), Historians Fallacy (Judging a person’s decision in the light of new information not available at the time.), Willed Ignorance (Refusing to change one’s mind or consider conflicting information based on a desire to maintain one’s existing beliefs.), Appeal to Coincidence (Concluding that a result is due to chance when the evidence strongly suggests otherwise.), Argument from Incredulity (Concluding that because you can’t or refuse to believe something, it must not be true, improbable, or the argument must be flawed.), Argument by Pigheadedness (This is a refusal to accept a well-proven argument for one of many reasons related to stubbornness.), Argument by Repetition (Repeating an argument or a premise over and over again in place of better supporting evidence.), Definist Fallacy (Defining a term in such a way that makes one’s position much easier to defend), Limited Scope (The theory doesn’t explain anything other than the phenomenon it explains (that one thing), and at best, is likely to be incomplete.), Moving the Goalposts (Demanding from an opponent that he or she address more and more points after the initial counter-argument has been satisfied refusing to concede or accept the opponent’s argument.), Argument from Fallacy (Concluding that the truth value of an argument is false based on the fact that the argument contains a fallacy.), False Dilemma (When only two choices are presented yet more exist, or a spectrum of possible choices exists between two extremes.), Argument from Ignorance (The assumption of a conclusion or fact based primarily on lack of evidence to the contrary.), Alternative Advance (When one is presented with just two choices, both of which are essentially the same, just worded differently.), Shifting of the Burden of Proof (Making a claim that needs justification, then demanding that the opponent justifies the opposite of the claim.), Proving Non Existence (Demanding that one proves the non-existence of something in place of providing adequate evidence for the existence of that something.), Proof Surrogate (A claim masquerading as proof or evidence, when no such proof or evidence is actually being offered.), Rationalization (Offering false or inauthentic excuses for our claim because we know the real reasons are much less persuasive or more embarrassing to share, or harsher than the manufactured ones given.), Spin Doctoring (Presenting information in a deceptive way that results in others interpreting the information in such a way that does not reflect reality but is how you want the information to be interpreted.), Lying with Statistics (Presenting statistical data in a very biased way.), Ad Hoc Rescue (When we desperately want to be right and hold on to certain beliefs, we begin to make up excuses without no real evidence as to why our belief could be.), False Attribution (Appealing to an irrelevant, unqualified, unidentified, biased, or fabricated source in support of an argument (modern usage).), Amazing Familiarity (The argument contains information that seems impossible to have obtained—like it came from an omniscient author.), Ludic Fallacy (Assuming flawless statistical models apply to situations where they actually don’t.), Missing Data Fallacy (Refusing to admit ignorance to the hypothesis and/or the conclusion, but insisting that your ignorance has to do with missing data that validate both the hypothesis and conclusion.), Begging the Question (Any form of argument where the conclusion is assumed in one of the premises.), Complex Question Fallacy (A question that has a presupposition built in, which implies something but protects the one asking the question from accusations of false claims.), Package Deal Fallacy (Assuming things that are often grouped together must always be grouped together, or the assumption that the ungrouping will have significantly more severe effects than anticipated.), Subjectivist Fallacy (Claiming something is true for one person, but not for someone else when, in fact, it is true for everyone (objective) as demonstrated by empirical evidence.), Distinction Without a Difference (The assertion that a position is different from another position based on the language when, in fact, both positions are the same – at least in practice or practical terms.), Hypothesis Contrary to Fact (Offering a poorly supported claim about what might have happened in the past or future, if (the hypothetical part) circumstances or conditions were different.), Shoehorning (The process of force-fitting some current affair into one’s personal, political, or religious agenda.), Appeal to Self-evident Truth (Making the claim that something is "self-evident" when it is not self-evident in place of arguing a claim with reason.), Subverted Support (The attempt to explain some phenomenon that does not actually occur or there is no evidence that it does.), Double Standard (Judging two situations by different standards when, in fact, you should be using the same standard.), Fantasy Projection (Confusing subjective experiences, usually very emotionally charged, with objective reality, then suggesting or demanding that others accept the subjective experience as objective reality.), Argument to Moderation (Asserting that given any two positions, there exists a compromise between them that must be correct.), Broken Window Fallacy (The illusion that destruction and money spent in recovery from destruction, is a net-benefit to society.), Self Sealing Argument (An argument or position is self-sealing if and only if no evidence can be brought against it no matter what.), Unfalsifiability (Confidently asserting that a theory or hypothesis is true or false even though the theory or hypothesis cannot possibly be contradicted by an observation or the outcome of any physical experiment, usually without strong evidence or good reasons.), Conspiracy Theory (Explaining that your claim cannot be proven or verified because the truth is being hidden and/or evidence destroyed by a group of two or more people.), Confusing an Explanation with an Excuse (Treating an explanation of a fact as if it were a justification of the fact, a valid reason for the fact, or evidence for the fact.), Limited Depth (Failing to appeal to an underlying cause, and instead simply appealing to membership in a category.), Alleged Certainty (Asserting a conclusion without evidence or premises, through a statement that makes the conclusion appear certain when, in fact, it is not.) Which one do you think it would fall under? You can also go back and change your previous classification. Output the results in this format: "level_3_results:#{}#" in all lower cases with no "fallacy". Only output the level 3 classification, no explanations. Only output a classification from the level 3 fallacy list provided in the format requested.
"""

one_shot_prompt = """
If you have to choose one classification among all the level 3 labels, which one will you pick for this statement? 
Undistributed Middle (A formal fallacy in a categorical syllogism where the middle term, or the term that does not appear in the conclusion, is not distributed to the other two terms.), Exclusive Premises (A standard form categorical syllogism that has two negative premises either in the form of “no X are Y” or “some X are not Y”.), Fallacy of Four Terms (This fallacy occurs in a categorical syllogism when the syllogism has four terms rather than the requisite three.), Illicit Substitution of Identicals (A fallacy due to confusing the knowing of a thing (extension) with the knowing of it under all its various names or descriptions (intension).), Illicit Minor (Any form of a categorical syllogism in which the minor term is distributed in the conclusion, but not in the minor premise.), Illicit Major (Any form of a categorical syllogism in which the major term is distributed in the conclusion, but not in the major premise.), Negative Conclusion from Affirmative Premises (The conclusion of a standard form categorical syllogism is negative, but both of the premises are positive.), Affirmative Conclusion from a Negative Premise (The conclusion of a standard form categorical syllogism is affirmative, but at least one of the premises is negative). Affirming the Consequent (An error in formal logic where if the consequent is said to be true, the antecedent is said to be true, as a result.), Denying the Antecedent (It is a fallacy where in a standard if/then premise, the antecedent (what comes after the “if”) is made not true, then it is concluded that the consequent (what comes after the “then”) is not true.), Negating Antecedent and Consequent (The valid form of this argument is as follows: If P then Q. Therefore, if not-Q then not-P.), Commutation of Conditionals (Switching the antecedent and the consequent in a logical argument.), Affirming a Disjunct (Making the false assumption that when presented with an either/or possibility, that if one of the options is true that the other one must be false.), Denying a Conjunct (A formal fallacy in which the first premise states that at least one of the two conjuncts (antecedent and consequent) is false and concludes that the other conjunct must be true.) False Conversion (The formal fallacy where the subject and the predicate terms of the proposition are switched (conversion) in the conclusion, in a proposition that uses “all” in its premise (type “A” forms), or “some/not” (type “O” forms).), Unwarranted Contrast (Assuming that implicature means implication, when it logically does not.), Quantifier Shift Fallacy (A fallacy of reversing the order of two quantifiers.), Existential Fallacy (A formal logical fallacy, which is committed when a categorical syllogism employs two universal premises (“all”) to arrive at a particular (“some”) conclusion.), Fallacy of Every and All (When an argument contains both universal quantifiers and existential quantifiers (all, some, none, every) with different meanings, and the order of the quantifiers is reversed.), Illicit Contraposition (A formal fallacy where switching the subject and predicate terms of a categorical proposition, then negating each, results in an invalid argument form.) Gamblers Fallacy (Reasoning that, in a situation that is pure random chance, the outcome can be affected by previous outcomes), Hot Hand Fallacy (The irrational belief that if you win or lose several chance games in a row, you are either “hot” or “cold,” respectively, meaning that the streak is likely to continue and has to do with something other than pure probability), Conjunction Fallacy (This occurs when one estimates a conjunctive statement (this and that) to be more probable than at least one of its component statements.), Disjunction Fallacy (It occurs when one estimates a disjunctive statement (this or that) to be less probable than at least one of its component statements.) Argument of the Beard Fallacy (When one argues that no useful distinction can be made between two extremes, just because there is no definable moment or point on the spectrum where the two extremes meet.), Appeal to Extremes Fallacy (Erroneously attempting to make a reasonable argument into an absurd one, by taking the argument to the extremes.), Type Token Fallacy (The type-token fallacy is committed when a word can refer to either a type (representing an abstract descriptive concept) or a token (representing an object that instantiates a concept) and is used in a way that makes it unclear which it refers to.), Use Mention Error Fallacy (Confusing the word used to describe a thing, with the thing itself.), Reification Fallacy (When an abstraction is treated as if it were a concrete, real event or physical entity – when an idea is treated as if had a real existence.), Fake Precision Fallacy (Using implausibly precise statistics to give the appearance of truth and certainty, or using a negligible difference in data to draw incorrect inferences.), No True Scotsman Fallacy (When a universal (“all”, “every”, etc.) claim is refuted, rather than conceding the point or meaningfully revising the claim, the claim is altered by going from universal to specific, and failing to give any objective criteria for the specificity.), Contextomy Fallacy (Removing a passage from its surrounding matter in such a way as to distort its intended meaning.), Stolen Concept Fallacy (Requiring the truth of the something that you are simultaneously trying to disprove), Anthropomorphism Fallacy (The attributing of human characteristics and purposes to inanimate objects, animals, plants, or other natural phenomena, or to gods.), Accent Fallacy (When the meaning of a word, sentence, or entire idea is interpreted differently by changing where the accent falls.), Ambiguity Fallacy (When an unclear phrase with multiple definitions is used within the argument; therefore, does not support the conclusion.), Alphabet Soup Fallacy (The deliberate and excessive use of acronyms and abbreviations to appear more knowledgeable in the subject or confuse others.), Equivocation Fallacy (Using an ambiguous term in more than one sense, thus making an argument misleading.) and Modal Scope Fallacy (Making a formal argument invalid by confusing the scope of what is actually necessary or possible.), Inconsistency (In terms of a fallacious argument, two or more propositions are asserted that cannot both possibly be true.), Conflicting Conditions (When the argument is self-contradictory and cannot possibly be true.), and Kettle Logic (Making (usually) multiple, contradicting arguments, in an attempt to support a single point or idea.). Political Correctness Fallacy (It is the assumption or admission that two or more groups, individuals, or ideas of groups or individuals, are equal, of equal value, or both true, based on the recent phenomenon of political correctness), Appeal to Complexity Fallacy (Concluding that because you don’t understand something, it must not be true, it’s improbable, or the argument must be flawed.), Statement of Conversion Fallacy (Accepting the truth of a claim based on a conversion story without considering any evidence for the truth of the claim.), Appeal to the Moon Fallacy (Using the argument, “If we can put a man on the moon, we could...” as evidence for the argument.), Quantum Physics Fallacy (Using quantum physics in an attempt to support your claim, when in no way is your claim related to quantum physics.), fact to fiction fallacy (Attempting to support a narrative or argument with facts that don’t support the narrative or argument.), Non Sequitur Fallacy (Evidence or reason is irrelevant or adds very little support to the conclusion.), Inflation of Conflict (Reasoning that because authorities cannot agree precisely on an issue, no conclusions can be reached at all, and minimizing the credibility of the authorities, as a result.), Argument by Fast Talking (When fast talking is seen as intelligence and/or confidence in the truth of one’s argument; therefore, seen as evidence of the truth of the argument itself.), Appeal to Intuition (Evaluating an argument based on "intuition" or "gut feeling" that is unable to be articulated, rather than evaluating the argument using reason.), Appeal to Closure (Accepting evidence on the basis of wanting closure, or to be done with the issue.), Appeal to Definition (Using a dictionary’s limited definition of a term as evidence that term cannot have another meaning, expanded meaning, or even conflicting meaning.), Spiritual Fallacy (Insisting that something meant to be literal is actually “spiritual” as an explanation or justification for something that otherwise would not fit in an explanation.), gish gallop (Overwhelming an interlocutor with as many arguments as possible, without regard for accuracy or strength of the arguments.), Denying the Correlative (Introducing alternatives when, in fact, there are none.), Red Herring (Attempting to redirect the argument to another issue to which the person doing the redirecting can better respond.), Strawman Fallacy (Substituting a person’s actual position or argument with a distorted, exaggerated, or misrepresented version of the position of the argument.), Avoiding the Issue (When an arguer responds to an argument by not addressing the points of the argument), Logic Chopping (Using the technical tools of logic in an unhelpful and pedantic manner by focusing on trivial details instead of directly addressing the main issue in dispute.), Meaningless Question (Asking a question that cannot be answered with any sort of rational meaning.), Failure to Elucidate (When the definition is made more difficult to understand than the word or concept being defined.), Argument by Gibberish (When incomprehensible jargon or plain incoherent gibberish is used to give the appearance of a strong argument, in place of evidence or valid reasons to accept the argument.), Hypnotic Bait and Switch (Stating several uncontroversially true statements in succession, followed by a claim that the arguer wants the audience to accept as true.), Traitorous Critic Fallacy (Responding to criticism by attacking a person’s perceived favorability to an out-group or dislike to the in-group as the underlying reason for the criticism rather than addressing the criticism itself, and suggesting that they stay away from the issue and/or leave the in-group.), Having Your Cake (Making an argument, or responding to one, in such a way that it does not make it at all clear what your position is.), Appeal to Common Belief (When the claim that most or many people in general or of a particular group accept a belief as true is presented as evidence for the claim.), Appeal to Popularity (Using the popularity of a premise or proposition as evidence for its truthfulness.), Appeal to Common Sense (Asserting that your conclusion or facts are just “common sense” when, in fact, they are not.), Appeal to Common Folk (In place of evidence, attempting to establish a connection to the audience based on being a “regular person” just like each of them.), Appeal to Trust (The belief that if a source is considered trustworthy or untrustworthy, then any information from that source must be true or false, respectively.), Argument from Age (The misconception that previous generations had superior wisdom to modern man, thus conclusions that rely on this wisdom are seen accepted as true or more true than they actually are.), Appeal to Heaven (Asserting the conclusion must be accepted because it is the “will of God” or “the will of the gods”.), Appeal to Tradition (Using historical preferences of the people (tradition), either in general or as specific as the historical preferences of a single individual, as evidence that the historical preference is correct.), Etymological Fallacy (The assumption that the present-day meaning of a word should be/is similar to the historical meaning.), Genetic Fallacy (Basing the truth claim of an argument on the origin of its claims or premises.), Appeal to Celebrity (Accepting a claim of a celebrity based on his or her celebrity status, not on the strength of the argument.), Appeal to Authority (Insisting that a claim is true simply because a valid authority or expert on the issue said it was true, without any other supporting evidence offered.), Appeal to False Authority (Using an alleged authority as evidence in your argument when the authority is not really an authority on the facts relevant to the argument.), Argument from False Authority (When a person making a claim is presented as an expert who should be trusted when his or her expertise is not in the area being discussed.), Blind Authority Fallacy (Asserting that a proposition is true solely on the authority making the claim), Argument by Personal Charm (When an argument is made stronger by the personal characteristics of the person making the argument, often referred to as “charm”.), Argument to the Purse (Concluding that the truth value of the argument is true or false based on the financial status of the author of the argument or the money value associated with the truth.), Ad Hominem Circumstantial (Suggesting that the person who is making the argument is biased or predisposed to take a particular stance, and therefore, the argument is necessarily invalid.), Gadarene Swine Fallacy (The assumption that because an individual is not in formation with the group, that the individual must be the one off course.), Ad Hominem Tu quoque (Claiming the argument is flawed by pointing out that the one making the argument is not acting consistently with the claims of the argument.), Bulverism (It is the assumption and assertion that an argument is flawed or false because of the arguer’s characteristic associated with the arguer’s identity.), Righteousness Fallacy (Assuming that just because a person’s intentions are good, they have the truth or facts on their side.), Self Righteousness Fallacy (Assuming that just because your intentions are good, you have the truth or facts on your side.), Reductio ad Hitlerum (The attempt to make an argument analogous with Hitler or the Nazi party.), Ad Hominem Guilt by Association (When the source is viewed negatively because of its association with another person or group who is already viewed negatively.), Identity Fallacy (When one’s argument is evaluated based on their physical or social identity when the strength of the argument is independent of identity.), Appeal to Stupidity (Attempting to get the audience to devalue reason and intellectual discourse, or devaluing reason and intellectual discourse based on the rhetoric of an arguer.), Ad Hominem Abusive (Attacking the person making the argument, rather than the argument itself, when the attack on the person is completely irrelevant to the argument the person is making.), Ad Fidentia (Attacking the person’s self-confidence in place of the argument or the evidence.), appeal to loyalty (When one is either implicitly or explicitly encouraged to consider loyalty when evaluating the argument when the truth of the argument is independent of loyalty.), Appeal to Accomplishment (When the argument being made is sheltered from criticism based on the level of accomplishment of the one making the argument.), Scapegoating (Unfairly blaming an unpopular person or group of people for a problem or a person or group that is an easy target for such blame.), Fallacy of Opposition (Asserting that those who disagree with you must be wrong and not thinking straight, primarily based on the fact that they are the opposition.), Proof by Intimidation (Making an argument purposely difficult to understand in an attempt to intimidate your audience into accepting it, or accepting an argument without evidence or being intimidated to question the authority or a priori assumptions of the one making the argument.), Poisoning the Well (To prime the audience with adverse information about the opponent from the start, in an attempt to make your claim more acceptable or discount the credibility of your opponent’s claim.), Wishful Thinking (When the desire for something to be true is used in place of/or as evidence for the truthfulness of the claim.), Appeal to Faith (It is the assertion that one must have (the right kind of) faith in order to understand the argument.), Notable Effort (Accepting good effort as a valid reason to accept the truth of the conclusion, even though the effort is unrelated to the truth.), Prejudicial Language (Loaded or emotive terms used to attach value or moral goodness to believing the proposition.), Special Pleading (Applying standards, principles, and/or rules to other people or circumstances, while making oneself or certain circumstances exempt from the same critical criteria, without providing adequate justification.), If By Whiskey (A response to a question that is contingent on the questioner’s opinions and makes use of words with strong connotations.), Overextended Outrage (One or more statistically rare cases are implied to be the norm or the trend (without evidence) for the purpose of expressing or inciting outrage toward an entire group.), Appeal to Ridicule (Presenting the argument in such a way that makes the argument look ridiculous, usually by misrepresenting the argument or the use of exaggeration.), Argument by Emotive Language (Substituting facts and evidence with words that stir up emotion, with the attempt to manipulate others into accepting the truth of the argument.), Style Over Substance (When the arguer embellishes the argument with compelling language or rhetoric, and/or visual aesthetics), Appeal to Anger (When the emotions of anger, hatred, or rage are substituted for evidence in an argument.), Appeal to Pity (The attempt to distract from the truth of the conclusion by the use of pity.), Appeal to Emotion (This is the general category of many fallacies that use emotion in place of reason in order to attempt to win the argument.), Appeal to Flattery (When an attempt is made to win support for an argument, not by the strength of the argument, but by using flattery on those whom you want to accept your argument.), Appeal to Spite (Substituting spite (petty ill will or hatred with the disposition to irritate, annoy, or thwart) for evidence in an argument, or as a reason to support or reject a claim.), pragmatic fallacy (Claiming that something is true because the person making the claim has experienced, or is referring to someone who has experienced, some practical benefit from believing the thing to be true.), Appeal to Force (When force, coercion, or even a threat of force is used in place of a reason in an attempt to justify a conclusion.), Appeal to Fear (When fear, not based on evidence or reason, is being used as the primary motivator to get others to accept an idea, proposition, or conclusion.). Fallacy of Composition (Inferring that something is true of the whole from the fact that it is true of some part of the whole.), Fallacy of Division (Inferring that something is true of one or more of the parts from the fact that it is true of the whole.), Stereotyping the fallacy (The general beliefs that we use to categorize people, objects, and events while assuming those beliefs are accurate generalizations of the whole group.), Ecological Fallacy (The interpretation of statistical data where inferences about the nature of individuals are deduced from inference for the group to which those individuals belong.), Oversimplified Cause Fallacy (When a contributing factor is assumed to be the cause, or when a complex array of causal factors is reduced to a single cause.), Accident Fallacy (When an attempt is made to apply a general rule to all situations when clearly there are exceptions to the rule.), mcnamara fallacy (When a decision is based solely on quantitative observations (i.e.,metrics, hard data, statistics) and all qualitative factors are ignored.), Overwhelming Exception (A generalization that is technically accurate, but has one or more qualifications which eliminate so many cases that the resulting argument is significantly weaker than the arguer implies.), Reductio ad Absurdum (A mode of argumentation or a form of argument in which a proposition is disproven by following its implications logically to an absurd conclusion), Nirvana Fallacy (Comparing a realistic solution with an idealized one, and discounting or even dismissing the realistic solution.), Relative Privation (Trying to make a scenario appear better or worse by comparing it to the best or worst case scenario.), imposter fallacy (When one suggests or claims, with insufficient evidence, that the group outliers who are viewed as damaging to the group are primarily made up of infiltrators of another group with the purpose of making the infiltrated group look bad.), Misleading Vividness (A small number of dramatic and vivid events are taken to outweigh a significant amount of statistical evidence), Appeal to Possibility (When a conclusion is assumed not because it is probably true, but because it is possible that it is true, no matter how improbable.), Rights To Ought Fallacy (When one conflates a reason for one’s rights (constitutional or other) with what one should do.), Psychogenetic Fallacy (Inferring some psychological reason why an argument is made then assuming it is invalid, as a result.), Weak Analogy (When an analogy is used to prove or disprove an argument, but the analogy is too dissimilar to be effective, that is, it is unlike the argument more than it is like the argument.), Extended Analogy (Suggesting that because two things are alike in some way and one of those things is like something else, then both things must be like that "something else".), Appeal to Equality (An assertion is deemed true or false based on an assumed pretense of equality, where what exactly is "equal" is not made clear, and not supported by the argument.), False Equivalence (An argument or claim in which two completely opposing arguments appear to be logically equivalent when in fact they are not.), Galileo Fallacy (The claim that because an idea is forbidden, prosecuted, detested, or otherwise mocked, it must be true, or should be given more credibility.), Post Designation (Drawing a conclusion from correlations observed in a given sample, but only after the sample has already been drawn, and without declaring in advance what correlations the experimenter was expecting to find.), Just In Case Fallacy (Making an argument based on the worst-case scenario rather than the most probable scenario, allowing fear to prevail over reason.), Selective Attention (Focusing your attention on certain aspects of the argument while completely ignoring or missing other parts.), nutpicking fallacy (When someone presents an atypical or weak member of a group as if they are a typical or strong representative.), Biased Sample Fallacy (Drawing a conclusion about a population based on a sample that is biased, or chosen in order to make it appear the population on average is different than it actually is.), Survivorship Fallacy (In its general form, the survivorship fallacy is basing a conclusion on a limited number of "winner" testimonies due to the fact we cannot or do not hear the testimonies of the losers.), Spotlight Fallacy (Assuming that the media’s coverage of a certain class or category is representative of the class or category in whole.), Hasty Generalization (Drawing a conclusion based on a small sample size, rather than looking at statistics that are much more in line with the typical or average situation.), Incomplete Comparison (An incomplete assertion that cannot possibly be refuted. This is popular in advertising.), Texas Sharpshooter Fallacy (Ignoring the difference while focusing on the similarities, thus coming to an inaccurate conclusion.), Faulty Comparison (Comparing one thing to another that is really not related, in order to make one thing look more or less desirable than it really is.), Base Rate Fallacy (Ignoring statistical information in favor of using irrelevant information, that one incorrectly believes to be relevant, to make a judgment.), Least Plausible Hypothesis (Choosing more unreasonable explanations for phenomena over more defensible ones.), Far Fetched Hypothesis (Offering a bizarre (far-fetched) hypothesis as the correct explanation without first ruling out more mundane explanations.), Cherry Picking (When only select evidence is presented in order to persuade the audience to accept a position, and evidence that would go against the position is withheld.), Argument by Selective Reading (When a series of arguments or claims is made and the opponent acts as if the weakest argument was the best one made.), deceptive sharing (Sharing an article, post, or meme on social media with the intent to influence public perception to perceive a statistically rare event as a common event.), Multiple Comparisons Fallacy (Claiming that unexpected trends that occur through random chance alone in a data set with a large number of variables are meaningful.), Magical Thinking (Making causal connections or correlations between two events not based on logic or evidence, but primarily based on superstition.), Slippery Slope (When a relatively insignificant first event is suggested to lead to a more significant event, which in turn leads to a more significant event, and so on, until some ultimate, significant event is reached, where the connection of each event is not only unwarranted but with each step it becomes more and more improbable.), Sunk Cost Fallacy (Reasoning that further investment is warranted on the fact that the resources already invested will be lost otherwise, not taking into consideration the overall losses involved in the further investment.), Jumping to Conclusions (Drawing a conclusion without taking the needed time to evaluate the evidence or reason through the argument.), Argument from Silence (Drawing a conclusion based on the silence of the opponent, when the opponent is refusing to give evidence for any reason.), Argument from Hearsay (Presenting the testimony of a source that is not an eyewitness to the event in question.), Anonymous Authority (When an unspecified source is used as evidence for the claim.), Insignificant Cause (An explanation that posits one minor factor, out of several that contributed, as its sole cause.), Just Because Fallacy (Refusing to respond to give reasons or evidence for a claim by stating yourself as the ultimate authority on the matter.), Appeal to the Law (When following the law is assumed to be the morally correct thing to do, without justification, or when breaking the law is assumed to be the morally wrong thing to do, without justification.), Appeal to Normality (Using social norms to determine what is good or bad.), False Effect (Claiming that the cause is true or false based on what we know about the effect in a claim of causality that has not been properly established.), Appeal to Consequences (Concluding that an idea or proposition is true or false because the consequences of it being true or false are desirable or undesirable.), Retrogressive Causation (Invoking the cause to eliminate the effect, or calling on the source to relieve the effect of the source.), Confusing Currently Unexplained with Unexplainable (Making the assumption that what cannot currently be explained is, therefore, unexplainable (impossible to explain).), Appeal to Desperation (Arguing that your conclusion, solution, or proposition is right based on the fact that something must be done, and your solution is "something."), Regression Fallacy (Ascribing a cause where none exists in situations where natural fluctuations exist while failing to account for these natural fluctuations.), Causal Reductionism (Assuming a single cause or reason when there were actually multiple causes or reasons.), Questionable Cause (Concluding that one thing caused another, simply because they are regularly associated.). Hedging (Refining your claim simply to avoid counter evidence and then acting as if your revised claim is the same as the original.), Circular Definition (A circular definition is defining a term by using the term in the definition), Homunculus Fallacy (An argument that accounts for a phenomenon in terms of the very phenomenon that it is supposed to explain, which results in an infinite regress.), Circular Reasoning (A type of reasoning in which the proposition is supported by the premises, which is supported by the proposition.), Tokenism (Interpreting a token gesture as an adequate substitute for the real thing.), Appeal to Novelty (Claiming that something that is new or modern is superior to the status quo, based exclusively on its newness.), Two Wrongs Make a Right (When a person attempts to justify an action against another person because the other person did take or would take the same action against him or her.), Appeal to Nature (When used as a fallacy, the belief or suggestion that “natural” is better than “unnatural” based on its naturalness.), Naturalistic Fallacy (When the conclusion expresses what ought to be, based only on what is, or what ought not to be, based on what is not.), Moralistic Fallacy (When the conclusion expresses what is, based only on what one believes ought to be, or what isn’t is based on what one believes ought not to be.), Suppressed Correlative (The attempt to redefine a correlative (one of two mutually exclusive options) so that one alternative encompasses the other, i.e. making one alternative impossible.), Historians Fallacy (Judging a person’s decision in the light of new information not available at the time.), Willed Ignorance (Refusing to change one’s mind or consider conflicting information based on a desire to maintain one’s existing beliefs.), Appeal to Coincidence (Concluding that a result is due to chance when the evidence strongly suggests otherwise.), Argument from Incredulity (Concluding that because you can’t or refuse to believe something, it must not be true, improbable, or the argument must be flawed.), Argument by Pigheadedness (This is a refusal to accept a well-proven argument for one of many reasons related to stubbornness.), Argument by Repetition (Repeating an argument or a premise over and over again in place of better supporting evidence.), Definist Fallacy (Defining a term in such a way that makes one’s position much easier to defend), Limited Scope (The theory doesn’t explain anything other than the phenomenon it explains (that one thing), and at best, is likely to be incomplete.), Moving the Goalposts (Demanding from an opponent that he or she address more and more points after the initial counter-argument has been satisfied refusing to concede or accept the opponent’s argument.), Argument from Fallacy (Concluding that the truth value of an argument is false based on the fact that the argument contains a fallacy.), False Dilemma (When only two choices are presented yet more exist, or a spectrum of possible choices exists between two extremes.), Argument from Ignorance (The assumption of a conclusion or fact based primarily on lack of evidence to the contrary.), Alternative Advance (When one is presented with just two choices, both of which are essentially the same, just worded differently.), Shifting of the Burden of Proof (Making a claim that needs justification, then demanding that the opponent justifies the opposite of the claim.), Proving Non Existence (Demanding that one proves the non-existence of something in place of providing adequate evidence for the existence of that something.), Proof Surrogate (A claim masquerading as proof or evidence, when no such proof or evidence is actually being offered.), Rationalization (Offering false or inauthentic excuses for our claim because we know the real reasons are much less persuasive or more embarrassing to share, or harsher than the manufactured ones given.), Spin Doctoring (Presenting information in a deceptive way that results in others interpreting the information in such a way that does not reflect reality but is how you want the information to be interpreted.), Lying with Statistics (Presenting statistical data in a very biased way.), Ad Hoc Rescue (When we desperately want to be right and hold on to certain beliefs, we begin to make up excuses without no real evidence as to why our belief could be.), False Attribution (Appealing to an irrelevant, unqualified, unidentified, biased, or fabricated source in support of an argument (modern usage).), Amazing Familiarity (The argument contains information that seems impossible to have obtained—like it came from an omniscient author.), Ludic Fallacy (Assuming flawless statistical models apply to situations where they actually don’t.), Missing Data Fallacy (Refusing to admit ignorance to the hypothesis and/or the conclusion, but insisting that your ignorance has to do with missing data that validate both the hypothesis and conclusion.), Begging the Question (Any form of argument where the conclusion is assumed in one of the premises.), Complex Question Fallacy (A question that has a presupposition built in, which implies something but protects the one asking the question from accusations of false claims.), Package Deal Fallacy (Assuming things that are often grouped together must always be grouped together, or the assumption that the ungrouping will have significantly more severe effects than anticipated.), Subjectivist Fallacy (Claiming something is true for one person, but not for someone else when, in fact, it is true for everyone (objective) as demonstrated by empirical evidence.), Distinction Without a Difference (The assertion that a position is different from another position based on the language when, in fact, both positions are the same – at least in practice or practical terms.), Hypothesis Contrary to Fact (Offering a poorly supported claim about what might have happened in the past or future, if (the hypothetical part) circumstances or conditions were different.), Shoehorning (The process of force-fitting some current affair into one’s personal, political, or religious agenda.), Appeal to Self-evident Truth (Making the claim that something is "self-evident" when it is not self-evident in place of arguing a claim with reason.), Subverted Support (The attempt to explain some phenomenon that does not actually occur or there is no evidence that it does.), Double Standard (Judging two situations by different standards when, in fact, you should be using the same standard.), Fantasy Projection (Confusing subjective experiences, usually very emotionally charged, with objective reality, then suggesting or demanding that others accept the subjective experience as objective reality.), Argument to Moderation (Asserting that given any two positions, there exists a compromise between them that must be correct.), Broken Window Fallacy (The illusion that destruction and money spent in recovery from destruction, is a net-benefit to society.), Self Sealing Argument (An argument or position is self-sealing if and only if no evidence can be brought against it no matter what.), Unfalsifiability (Confidently asserting that a theory or hypothesis is true or false even though the theory or hypothesis cannot possibly be contradicted by an observation or the outcome of any physical experiment, usually without strong evidence or good reasons.), Conspiracy Theory (Explaining that your claim cannot be proven or verified because the truth is being hidden and/or evidence destroyed by a group of two or more people.), Confusing an Explanation with an Excuse (Treating an explanation of a fact as if it were a justification of the fact, a valid reason for the fact, or evidence for the fact.), Limited Depth (Failing to appeal to an underlying cause, and instead simply appealing to membership in a category.), Alleged Certainty (Asserting a conclusion without evidence or premises, through a statement that makes the conclusion appear certain when, in fact, it is not.) Output the results in this format: "level_3_results:#{}#" in all lower cases with no "fallacy"
Only output the level 3 classification, no explanations. Only output a classification from the level 3 fallacy list provided in the format requested.
"""


### Set up Claude env

In [16]:
import anthropic

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key="",
)

### Claude Function Definitions and running

In [98]:
level_1_class = []
level_2_class = []
level_3_class = []
level_3_zero_shot = []
level_2_class_list = ["syllogism", "proposition", "quantification", "probability", "ambiguity", "inconsistency", "irrelevance", "insufficiency", "inappropriate presumption"]
def get_claude37_response(prompt):
    response = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        max_tokens=1024,
        messages=[{"role": "user", "content": prompt}],
        stream=False
    )
    return response.content[0].text

def get_level_1_classification(index):
    prompt = first_level_prompt_template + examples[index]
    response = get_claude37_response(prompt)
    level_1 = response.split("#")[1]
    level_1_class.append(level_1)
    return level_1

def get_level_2_classification(index):
    prompt = second_level_prompt_template + examples[index]
    response = get_claude37_response(prompt)
    level_2 = response.split("#")[1]
    if level_2 in level_2_class_list:
        level_2_class.append(level_2)
        return level_2
    else:
        print("Please try again and only output a classification from the level 2 fallacy list provided in the format requested.")
        return get_level_2_classification(index)
        
def get_level_3_classification(level_2, index):
    print("level 3 classification for example " + str(index))
    prompt = ""
    if level_2 == "syllogism":
        prompt = third_level_prompt_template_syllogism
    elif level_2 == "proposition":
        prompt = third_level_prompt_template_proposition
    elif level_2 == "quantification":
        prompt = third_level_prompt_template_quantification
    elif level_2 == "probability":
        prompt = third_level_prompt_template_probability
    elif level_2 == "ambiguity":
        prompt = third_level_prompt_template_ambiguity
    elif level_2 == "inconsistency":
        prompt = third_level_prompt_template_inconsistency
    elif level_2 == "irrelevance":
        prompt = third_level_prompt_template_irrelevance
    elif level_2 == "insufficiency":
        prompt = third_level_prompt_template_insufficiency
    elif level_2 == "inappropriate presumption":
        prompt = third_level_prompt_template_inappropriate_presumption
        
    prompt = prompt + examples[index]
    response = get_claude37_response(prompt)
    level_3 = response.split("#")[1]
    level_3_class.append(level_3)
    return level_3

def get_level_3_zero_shot(index):
    prompt = one_shot_prompt + examples[index]
    response = get_claude37_response(prompt)
    level_3 = response.split("#")[1]
    level_3_zero_shot.append(level_3)
    return level_3

def main():
    for i in range(194, len(examples)):
        level_1 = get_level_1_classification(i)
        level_2 = get_level_2_classification(i)
        level_3 = get_level_3_classification(level_2, i)
        level_3_zero_shot = get_level_3_zero_shot(i)

main()

level 3 classification for example 194
level 3 classification for example 195
level 3 classification for example 196
level 3 classification for example 197
level 3 classification for example 198
level 3 classification for example 199
level 3 classification for example 200
level 3 classification for example 201
level 3 classification for example 202
level 3 classification for example 203
level 3 classification for example 204
level 3 classification for example 205
level 3 classification for example 206
level 3 classification for example 207
level 3 classification for example 208
level 3 classification for example 209
level 3 classification for example 210
level 3 classification for example 211
level 3 classification for example 212
level 3 classification for example 213
level 3 classification for example 214
level 3 classification for example 215
level 3 classification for example 216
level 3 classification for example 217
level 3 classification for example 218
level 3 classification fo

### Set up ChatGPT env

In [ ]:
import openai

openai.api_key = ""


In [182]:
level_1_class = []
level_2_class = []
level_3_class = []
level_3_zero_shot = []
level_2_class_list = ["syllogism", "proposition", "quantification", "probability", "ambiguity", "inconsistency", "irrelevance", "insufficiency", "inappropriate presumption"]

def get_gpt_response(prompt):
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content


def get_level_1_classification(index):
    prompt = first_level_prompt_template + examples[index]
    response = get_gpt_response(prompt)
    level_1 = response.split("#")[1]
    level_1_class.append(level_1)
    return level_1

def get_level_2_classification(index):
    prompt = second_level_prompt_template + examples[index]
    response = get_gpt_response(prompt)
    level_2 = response.split("#")[1]
    if level_2 in level_2_class_list:
        level_2_class.append(level_2)
        return level_2
    else:
        print("Please try again and only output a classification from the level 2 fallacy list provided in the format requested.")
        return get_level_2_classification(index)
        
def get_level_3_classification(level_2, index):
    print("level 3 classification for example " + str(index))
    prompt = ""
    if level_2 == "syllogism":
        prompt = third_level_prompt_template_syllogism
    elif level_2 == "proposition":
        prompt = third_level_prompt_template_proposition
    elif level_2 == "quantification":
        prompt = third_level_prompt_template_quantification
    elif level_2 == "probability":
        prompt = third_level_prompt_template_probability
    elif level_2 == "ambiguity":
        prompt = third_level_prompt_template_ambiguity
    elif level_2 == "inconsistency":
        prompt = third_level_prompt_template_inconsistency
    elif level_2 == "irrelevance":
        prompt = third_level_prompt_template_irrelevance
    elif level_2 == "insufficiency":
        prompt = third_level_prompt_template_insufficiency
    elif level_2 == "inappropriate presumption":
        prompt = third_level_prompt_template_inappropriate_presumption
        
    prompt = prompt + examples[index]
    response = get_gpt_response(prompt)
    level_3 = response.split("#")[1]
    level_3_class.append(level_3)
    return level_3
    
def get_level_3_zero_shot(index):
    prompt = one_shot_prompt + examples[index]
    response = get_gpt_response(prompt)
    level_3 = response.split("#")[1]
    level_3_zero_shot.append(level_3)
    return level_3


def main():
    for i in range(193, len(examples)):
        level_1 = get_level_1_classification(i)
        level_2 = get_level_2_classification(i)
        level_3 = get_level_3_classification(level_2, i)
        level_3_zero_shot = get_level_3_zero_shot(i)

main()

level 3 classification for example 193
level 3 classification for example 194
level 3 classification for example 195
level 3 classification for example 196
level 3 classification for example 197
level 3 classification for example 198
level 3 classification for example 199
level 3 classification for example 200
level 3 classification for example 201
level 3 classification for example 202
level 3 classification for example 203
level 3 classification for example 204
level 3 classification for example 205
level 3 classification for example 206
level 3 classification for example 207
level 3 classification for example 208
level 3 classification for example 209
level 3 classification for example 210
level 3 classification for example 211
level 3 classification for example 212
level 3 classification for example 213
level 3 classification for example 214
level 3 classification for example 215
level 3 classification for example 216
Please try again and only output a classification from the level

### Set up Gemini env

In [ ]:
from google import genai
api_key=""

In [274]:
level_1_class = []
level_2_class = []
level_3_class = []
level_3_zero_shot = []
level_2_class_list = ["syllogism", "proposition", "quantification", "probability", "ambiguity", "inconsistency", "irrelevance", "insufficiency", "inappropriate presumption"]

def get_gemini_response(prompt):
    response = genai.Client(api_key=api_key).models.generate_content(
        model="gemini-2.5-flash-preview-05-20", contents=prompt)
    return response.text


def get_level_1_classification(index):
    prompt = first_level_prompt_template + examples[index]
    response = get_gpt_response(prompt)
    level_1 = response.split("#")[1]
    level_1_class.append(level_1)
    return level_1

def get_level_2_classification(index):
    prompt = second_level_prompt_template + examples[index]
    response = get_gpt_response(prompt)
    level_2 = response.split("#")[1]
    if level_2 in level_2_class_list:
        level_2_class.append(level_2)
        return level_2
    else:
        print("Please try again and only output a classification from the level 2 fallacy list provided in the format requested.")
        return get_level_2_classification(index)

def get_level_3_classification(level_2, index):
    print("level 3 classification for example " + str(index))
    prompt = ""
    if level_2 == "syllogism":
        prompt = third_level_prompt_template_syllogism
    elif level_2 == "proposition":
        prompt = third_level_prompt_template_proposition
    elif level_2 == "quantification":
        prompt = third_level_prompt_template_quantification
    elif level_2 == "probability":
        prompt = third_level_prompt_template_probability
    elif level_2 == "ambiguity":
        prompt = third_level_prompt_template_ambiguity
    elif level_2 == "inconsistency":
        prompt = third_level_prompt_template_inconsistency
    elif level_2 == "irrelevance":
        prompt = third_level_prompt_template_irrelevance
    elif level_2 == "insufficiency":
        prompt = third_level_prompt_template_insufficiency
    elif level_2 == "inappropriate presumption":
        prompt = third_level_prompt_template_inappropriate_presumption
        
    prompt = prompt + examples[index]
    response = get_gpt_response(prompt)
    level_3 = response.split("#")[1]
    level_3_class.append(level_3)
    return level_3
    
def get_level_3_zero_shot(index):
    prompt = one_shot_prompt + examples[index]
    response = get_gpt_response(prompt)
    level_3 = response.split("#")[1]
    level_3_zero_shot.append(level_3)
    return level_3


def main():
    for i in range(193, len(examples)):
        level_1 = get_level_1_classification(i)
        level_2 = get_level_2_classification(i)
        level_3 = get_level_3_classification(level_2, i)
        level_3_zero_shot = get_level_3_zero_shot(i)

main()

level 3 classification for example 193
level 3 classification for example 194
level 3 classification for example 195
level 3 classification for example 196
level 3 classification for example 197
level 3 classification for example 198
level 3 classification for example 199
level 3 classification for example 200
level 3 classification for example 201
level 3 classification for example 202
level 3 classification for example 203
level 3 classification for example 204
level 3 classification for example 205
level 3 classification for example 206
level 3 classification for example 207
level 3 classification for example 208
level 3 classification for example 209
level 3 classification for example 210
level 3 classification for example 211
level 3 classification for example 212
level 3 classification for example 213
level 3 classification for example 214
level 3 classification for example 215
level 3 classification for example 216
level 3 classification for example 217
level 3 classification fo

In [128]:
level_1_class

['informal',
 'informal',
 'informal',
 'informal',
 'informal',
 'informal',
 'informal',
 'informal',
 'formal',
 'formal',
 'formal']

In [130]:
level_2_class

['ambiguity',
 'inappropriate presumption',
 'insufficiency',
 'inappropriate presumption',
 'irrelevance',
 'irrelevance',
 'irrelevance',
 'inconsistency',
 'syllogism',
 'syllogism',
 'syllogism']

In [132]:
level_3_class

['contextomy',
 'moralistic',
 'appeal to possibility',
 'willed ignorance',
 'ad hominem abusive',
 'ad hominem circumstantial',
 'ad hominem guilt by association',
 'inconsistency',
 'exclusive premises',
 'illicit_minor']

In [134]:
level_3_zero_shot

['contextomy',
 'moralistic reasoning',
 'appeal to possibility',
 'wishful thinking',
 'ad_hominem_circumstantial',
 'ad hominem circumstantial',
 'ad hominem guilt by association',
 'ad_hominem_tu_quoque',
 'exclusive premises',
 'false_dilemma']

### Print out all examples and their classifications on different levels

In [276]:
for i in range(len(level_3_zero_shot)):
    print(examples[193 + i])
    print(level_1_class[i])
    print(level_2_class[i])
    print(level_3_class[i])
    print(level_3_zero_shot[i])
#example 42 - I shop at Natural Happy Sunshine Store (NHSS), which is much better than your grocery store because at NHSS everything is natural including the 38-year-old store manager’s long gray hair and saggy breasts."

Example #1: Mike: It is morally wrong to cheat on your spouse, why on earth would you have done that? Ken: But what is morality exactly? Mike: It’s a code of conduct shared by cultures. Ken: But who creates this code?...

informal
irrelevance
avoiding the issue
avoid_the_issue
Example #1: I am going into surgery tomorrow so please pray for me.  If enough people pray for me, God will protect me from harm and see to it that I have a successful surgery and speedy recovery.

informal
inappropriate presumption
appeal to self-evident truth
appeal to heaven
Example #1: Peter Gibbons: It's NOT wrong. INITECH is wrong. INITECH is an evil corporation, all right? Chochkies is wrong. Doesn't it bother you that you have to get up in the morning and you have to put on a bunch of pieces of flair? Joanna: Yeah, but I'm not about to go in and start taking money from the register. Peter Gibbons: Well, maybe you should. You know, the Nazis had pieces of flair that they made the Jews wear. Joanna: What?



In [272]:
examples[193]

'Example #1: Mike: It is morally wrong to cheat on your spouse, why on earth would you have done that? Ken: But what is morality exactly? Mike: It’s a code of conduct shared by cultures. Ken: But who creates this code?...\n'